In [1]:
import pandas as pd
import statsmodels.api as sm
import pandas as pd

In [19]:
def get_logit(dist_goal_df):
    dist_goal_df['const'] = 1
    X = dist_goal_df[['const', 'distance']]
    y = dist_goal_df['is_goal']
    logit_model = sm.Logit(y, X)
    logit_result = logit_model.fit()
    return logit_result

In [20]:
def get_working_dataframe(df_base):
    working_df = df_base[['arenaAdjustedShotDistance', 'goal']]
    working_df = working_df.rename(columns={'arenaAdjustedShotDistance': 'distance', 'goal': 'is_goal'})
    return working_df

In [21]:
def get_base_dataframe():
    file_path = 'shots_2022.csv'
    df_base = pd.read_csv(file_path)
    return df_base

In [22]:
def run_test_case(logit, distance):
    test_case_const = 1
    test_case = {'const': test_case_const, 'distance': distance}
    test_case_df = pd.DataFrame(test_case, index=[0])
    predicted_probability = logit.predict(test_case_df)
    predicted_class = (predicted_probability >= 0.5).astype(int)
    print(f"Predicted Probability: {predicted_probability.iloc[0]:.4f}")
    print(f"Predicted Class: {predicted_class.iloc[0]}")

In [24]:
base_df = get_base_dataframe()
working_df = get_working_dataframe(base_df)
logit = get_logit(working_df)
run_test_case(logit,25)

Optimization terminated successfully.
         Current function value: 0.244490
         Iterations 7
Predicted Probability: 0.0809
Predicted Class: 0


In [8]:
import joblib
joblib.dump(logit, 'hockey_logit.joblib')

['hockey_logit.joblib']

In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)
model = joblib.load('hockey_logit.joblib')
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        features = data['features']
        prediction = model.predict([features])[0]
        return jsonify({'prediction': prediction})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)